In [1]:
import plotly.io as pio
pio.renderers.default="iframe"

# Data Visualization using SQL

set default to iframe for quarto readability

### Databases

Today we are making a database. Similar to the pandas dataframe, we can create a dataframe by using the connect method and then inserting the name of the database.

We also make some dataframes which will be added to the dataframe shortly

In [2]:
import pandas as pd
temps = pd.read_csv("temps.csv")



import sqlite3
conn = sqlite3.connect("temps.db")  # create a database in current directory called temps.db



df = pd.read_csv("temps.csv")






countries_url = "https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"
countries = pd.read_csv(countries_url)
countries.head(10)


,FIPS 10-4,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania
3,AG,DZ,Algeria
4,AQ,AS,American Samoa
5,AN,AD,Andorra
6,AO,AO,Angola
7,AV,AI,Anguilla
8,AY,AQ,Antarctica
9,AC,AG,Antigua and Barbuda



#### Prepping the dataframe

This simply renames and sorts the dataframe such that it is more readable and better accessible when we use it later

In [3]:
def prepare_df(df):
    df = df.set_index(keys=["ID", "Year"])
    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)

temps = prepare_df(temps)
temps.head()

,ID,Year,Month,Temp
0,ACW00011604,1961,1,-0.89
1,ACW00011604,1961,2,2.36
2,ACW00011604,1961,3,4.72
3,ACW00011604,1961,4,7.73
4,ACW00011604,1961,5,11.28


#### Add to database

to_sql converts a dataframe to a database
we first give the database table name, then the database name, then tell it to "replace" if such object already exists



temps.to_sql("temperatures", conn, if_exists = "replace", index = False)



We do the same to "stations" and "countries"

In [4]:

url = "station-metadata.csv"
stations = pd.read_csv(url)
stations.to_sql("stations", conn, if_exists = "replace", index=False)


27585

In [5]:
countries.to_sql("countries", conn, if_exists = "replace", index = False)

279

It is good practice to close databases when not in use

In [6]:
conn.close()

## Create a query using sql

we can convert a database to a pandas dataframe by using pd.read_sql_query.
In that method, we hand it a command and the database name.
the command, cmd tells the function what variables to include and how to include them

In [7]:

def query_climate_database(country, year_begin, year_end, month):
    """creates a dataframe of temperature of a certain country from a certain month within a range of years

    Keyword arguments:
    country -- string: name of the country to be analyzed
    year_begin -- int: beginning range of year
    year_end -- int: end range of year
    month -- int: month from 1-12
    """

    cmd = \
    f"""
    SELECT S.name, S.latitude, S.longitude, C.name, T.year, T.month, T.temp
    FROM temperatures T 
    LEFT JOIN stations S ON T.id = S.id
    LEFT JOIN countries C ON SUBSTRING(T.id,1, 2) = C.`FIPS 10-4` 
    WHERE T.year >= {year_begin} AND T.year <= {year_end} AND T.month == {month} AND C.name == "{country}"
    """
    return pd.read_sql_query(cmd, conn)




Here we reopen the database and call the function

In [8]:
conn = sqlite3.connect("temps.db")

USAtemps = query_climate_database("United States", 1990, 2020, 10)
USAtemps

,NAME,LATITUDE,LONGITUDE,Name,Year,Month,Temp
0,ADDISON,34.2553,-87.1814,United States,2001,10,13.83
1,ADDISON,34.2553,-87.1814,United States,2002,10,17.11
2,ADDISON,34.2553,-87.1814,United States,2004,10,17.98
3,ADDISON,34.2553,-87.1814,United States,2005,10,14.91
4,ADDISON,34.2553,-87.1814,United States,2010,10,16.04
...,...,...,...,...,...,...,...
202900,LINCOLN_11_SW,40.6953,-96.8542,United States,2016,10,14.42
202901,LINCOLN_11_SW,40.6953,-96.8542,United States,2017,10,12.52
202902,LINCOLN_11_SW,40.6953,-96.8542,United States,2018,10,10.08
202903,LINCOLN_11_SW,40.6953,-96.8542,United States,2019,10,9.31


## Function to plot average yearly termperature change in a certain country  


This is a helper function to calculate **year-over-year average change in temperature**

In [9]:
from sklearn.linear_model import LinearRegression

def coef(data_group):
    x = data_group[["Year"]] # 2 brackets because X should be a df
    y = data_group["Temp"]   # 1 bracket because y should be a series
    LR = LinearRegression()
    LR.fit(x, y)
    return LR.coef_[0]

Function which calculates plots the data using a plotly scatter_mapbox

In [10]:
from plotly import express as px
import numpy as np
def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    countryTemps = query_climate_database(country, year_begin, year_end, month)
    
    #removes enteries with less than min_obs observations
    value_counts = countryTemps["NAME"].value_counts()
    countryTemps = countryTemps[countryTemps['NAME'].isin(value_counts.index[value_counts >= min_obs])]
    
    
    #calculate change in temperature
    coefs = countryTemps.groupby(["NAME"]).apply(coef)
    coefs = coefs.reset_index()
    
    #add the change in temperature to the main dataframe
    countryTemps = pd.merge(coefs, countryTemps, on = ["NAME"])
    countryTemps.rename(columns = {0: "Yearly Temperature Increase"}, inplace = True) 
    
    colorMap = px.colors.diverging.RdGy_r
    #dictionary to translate month number to month name
    monthKey = { 1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"}
    
    
    fig = px.scatter_mapbox(countryTemps,
                            lat = "LATITUDE",
                            lon = "LONGITUDE",
                            hover_name = "NAME",
                            color = "Yearly Temperature Increase",
                            mapbox_style = "carto-positron",
                            color_continuous_scale = colorMap,
                            zoom = 2.5,
                            title = f"Yearly Increase in Temperature (°C) in {monthKey[month]} for stations in {country} from {year_begin} to {year_end}",
                            **kwargs
                            )
    fig.update_layout(
        autosize=False,
        width=1000,
        height=750)
    return fig


Notice this map is interactive

In [11]:
temperature_coefficient_plot("United States", 1980, 2020, 1, 20)


## Comparing climate of two different countries

Now we will create a function which compares the temperature of two countries over the years

The first function gets a dataframe from the function used earlier
The second function combines 2 dataframes and plots them in a line graph

In [12]:
def countryYearlyTempChange(country, year_begin, year_end, month, min_obs):
    countryTemps = query_climate_database(country, year_begin, year_end, month)
    
    #removes enteries with less than min_obs observations
    value_counts = countryTemps["NAME"].value_counts()
    countryTemps = countryTemps[countryTemps['NAME'].isin(value_counts.index[value_counts >= min_obs])]
    
    return countryTemps
   
def countryCompareLineGraph(country1, country2, year_begin, year_end, month, min_obs, **kwargs):
    country1 = countryYearlyTempChange(country1, year_begin, year_end, month, min_obs)
    country2 = countryYearlyTempChange(country2, year_begin, year_end, month, min_obs)
    
    df = pd.concat([country1, country2], ignore_index = True)
    
    
    fig = px.scatter(df, x="Year", y="Temp", color='Name')
    fig.show()


In [13]:
countryCompareLineGraph("United States", "India", 1990, 2010, 10, 20)

## Temperature by year for a country

To get a grasp at climate change, one can see the climate by year in this graph"

In [14]:
def countryTemp (country, year_begin, year_end, month, min_obs):
    countryTemps = query_climate_database(country, year_begin, year_end, month)
    
    #removes enteries with less than min_obs observations
    value_counts = countryTemps["NAME"].value_counts()
    countryTemps = countryTemps[countryTemps['NAME'].isin(value_counts.index[value_counts >= min_obs])]
    
    
    monthKey = { 1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"}
    
    fig = px.bar(countryTemps, x ='Year', y='Temp', title = f"Average temperature in {monthKey[month]} in {country}")
    fig.show()

In [15]:
countryTemp('United States', 1990, 2020, 10, 20)